In [2]:
import numpy as np
import dill
import pandas as pd

import datetime as dt
from tqdm import tqdm
from collections import defaultdict, Counter

import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")

# pd.set_option('display.max_rows', 3000)
# pd.reset_option('display.max_rows')

In [4]:
df = pd.read_csv('C:/Users/cheny/Desktop/UTI_ML/wln/New_MIMIC4_p_1.csv')
date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME']
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.shape

(67207, 373)

### 整合字典

In [7]:
#ID对应的字典
D_iv_ITEMS = pd.read_csv('C:/MIMIC IV/3.1/icu/d_items.csv.gz')
D_iv_ITEMS.columns = D_iv_ITEMS.columns.str.upper()

D_iii_ITEMS = pd.read_csv('C:/MIMIC III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/d_items.csv.gz')
D_iii_ITEMS.columns = D_iii_ITEMS.columns.str.upper()

print(D_iii_ITEMS.shape,D_iv_ITEMS.shape)

D_ITEMS = pd.concat([D_iii_ITEMS[['ITEMID', 'LABEL', 'LINKSTO', 'CATEGORY', 'UNITNAME','PARAM_TYPE']],D_iv_ITEMS[['ITEMID', 'LABEL', 'LINKSTO', 'CATEGORY', 'UNITNAME','PARAM_TYPE']]])
D_ITEMS['CATEGORY'] = D_ITEMS['CATEGORY'].str.upper()
print(D_ITEMS.shape)

D_ITEMS = D_ITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
D_ITEMS = D_ITEMS.dropna(subset=['ITEMID','LABEL'])
D_ITEMS = D_ITEMS.sort_values(by='LABEL')
D_ITEMS = D_ITEMS.drop_duplicates(subset=['ITEMID'],keep='first')
print(D_ITEMS.shape)

(12487, 10) (4095, 9)
(16582, 6)
(13610, 6)


In [9]:
D_iv_LABITEMS = pd.read_csv('C:/MIMIC IV/3.1/hosp/D_LABITEMS.csv.gz')
D_iv_LABITEMS.columns = D_iv_LABITEMS.columns.str.upper()

D_iii_LABITEMS = pd.read_csv('C:/MIMIC III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/D_LABITEMS.csv.gz')
D_iii_LABITEMS.columns = D_iii_LABITEMS.columns.str.upper()

print(D_iii_LABITEMS.shape,D_iv_LABITEMS.shape)

D_LABITEMS = pd.concat([D_iii_LABITEMS[['ITEMID', 'LABEL', 'CATEGORY']],D_iv_LABITEMS[['ITEMID', 'LABEL', 'CATEGORY']]])
D_LABITEMS['CATEGORY'] = D_LABITEMS['CATEGORY'].str.upper()

print(D_LABITEMS.shape)
D_LABITEMS = D_LABITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
D_LABITEMS['LABEL'].replace(' ', np.nan, inplace=True)
D_LABITEMS['LABEL'].replace('(Albumi','(Albumin)', inplace=True)
D_LABITEMS = D_LABITEMS.dropna(subset=['ITEMID','LABEL'])
D_LABITEMS = D_LABITEMS.sort_values(by='LABEL')
D_LABITEMS = D_LABITEMS.drop_duplicates(subset=['ITEMID'],keep='first')
D_LABITEMS['LINKSTO'] = 'chartevents'
print(D_LABITEMS.shape)

(753, 6) (1650, 4)
(2403, 3)
(1718, 4)


In [11]:
All_ITEMS = pd.concat([D_ITEMS[['ITEMID', 'LABEL', 'LINKSTO','CATEGORY']], D_LABITEMS])
All_ITEMS = All_ITEMS.sort_values(by='LABEL')
All_ITEMS = All_ITEMS[~(All_ITEMS.CATEGORY == 'ADT')]
del(D_ITEMS,D_LABITEMS)
print(All_ITEMS.shape)

(15317, 4)


In [13]:
del(D_iii_ITEMS,D_iv_ITEMS,D_iii_LABITEMS,D_iv_LABITEMS)

### 1. 把变形的进行整合

In [16]:
# # 查找变形的同意思的label, 如果已经知道则不用做
# labels = All_ITEMS['LABEL'].dropna().unique()
# labels_lower = [lbl.lower() for lbl in labels]

# matches = []

# for i, lbl1 in enumerate(labels_lower):
#     for j, lbl2 in enumerate(labels_lower):
#         if lbl1 in lbl2:
#             matches.append((labels[i], labels[j]))  # 保留原始大小写版本

# overlap_df = pd.DataFrame(matches, columns=['Sub_Label', 'Full_Label'])

# grouped = overlap_df.groupby('Sub_Label')['Full_Label'].count().reset_index(name='count')
# single_match = grouped[grouped['count'] == 1]['Sub_Label']
# to_check = overlap_df[overlap_df['Sub_Label'].isin(single_match)]
# duplicates = to_check[to_check['Sub_Label'] == to_check['Full_Label']]
# overlap_df = overlap_df.drop(index=duplicates.index).reset_index(drop=True)

# overlap_df = (
#     overlap_df.groupby('Sub_Label')['Full_Label']
#       .agg(list)
#       .reset_index()
# )

# # 将 Full_Label 列转换为元组以便比较
# overlap_df['Full_Label_tuple'] = overlap_df['Full_Label'].apply(tuple)

# # 删除 Full_Label 列中内容重复的行，只保留第一行
# overlap_df = overlap_df.drop_duplicates(subset='Full_Label_tuple', keep='first')

# # 删除辅助列，恢复原来的 DataFrame 格式
# overlap_df = overlap_df.drop(columns='Full_Label_tuple')
# overlap_df.shape

In [18]:
Albumin = ['ALBUMIN','(Albumin)','<Albumin>','Albumin','albumin']
ALT = ['ALT','Alanine Aminotransferase (ALT)']
AST = ['AST','Asparate Aminotransferase (AST)']
PLT = ['Platelet','Platelet Count', 'Platelets', 'PLTS']
Sodium = ['Sodium','Sodium (135-148)']
Sodium_WholeBlood = ['Sodium, Whole Blood']
Magnesium = ['Magnesium', 'Magnesium (1.6-2.6)']
Chloride = ['Chloride', 'Chloride (100-112)']
Phosphorous = ['Phosphorous','Phosphorous(2.7-4.5)']
INR = ['INR','INR (2-4 ref. range)']

NSBP = ['NBP [Systolic]','Non Invasive Blood Pressure systolic']
NDBP = ['NBP [Diastolic]','Non Invasive Blood Pressure diastolic']
NMBP = ['NBP Mean','Non Invasive Blood Pressure mean']

MAP = ["ART BP Mean", "ART MAP", "ART MEAN", "Arterial BP Mean", 
 "Arterial BP Mean #2", "Arterial Blood Pressure mean", 
 "FEM ART MAP", "FEM ART MEAN", "art mean", "arterial mean #2"]
ABP = ["ART BP", "ART Blood Pressure", "Arterial BP", "Arterial Blood Pressure", 
 "FEM ART BP", "Fem Art BP", "L FEM ART BP", "L fem art bp", 
 "R fem art BP", "RAD ART BP", "RT FEMORAL ART BP"]
ASBP = ["ART BP Systolic", "Arterial BP #2 [Systolic]", 
        "Arterial BP [Systolic]", "Arterial Blood Pressure systolic"]
ADBP = ["ART BP Diastolic", "Arterial BP #2 [Diastolic]", 
 "Arterial BP [Diastolic]", "Arterial Blood Pressure diastolic"]

SaO2 = ["ART %O2 saturation (PA Line)", "Arterial O2 Saturation"]
SpO2 = ['Oxygen Saturation', 'PAR-Oxygen saturation','SpO2','O2 saturation pulseoxymetry']
PaO2 = ['Arterial PaO2','pO2','Arterial O2 pressure']

Arterial_pH = ["Art.pH", "Arterial pH", "pH (Art)", "pH (Arterial)"]
Urine_PH = ['Urine PH', 'Urine pH', 'urine PH', 'urine pH', 'urine ph']

Femoral_ArteryLine = ["FEM ART LINE", "L FEM ART LINE", "L fem art line", "femoral ART Line", 
'FEM ART',"R FEM ART LINE", "R fem art line", "RT FEM ART LINE",'fem art', 'femoral ART Line', 'r fem art line', 'rt art line']  

Arterial_PaCO2 = ['Arterial PaCO2', 'Arterial CO2 Pressure']
Arterial_TCO2 = ['Arterial CO2 (Calc)', 'TCO2 (calc) Arterial']

ETCO2 = ['ETCO2', 'EtCO2']

Pain = ['Pain','Pain (0-10)','Pain Level']
Pain_Presence = ['Currently experiencing pain','Pain Present']

Dialysis = ['Dialysis','DIALYSIS']
Dialysis_Output = ['DIALYSIS OUT', 'DIALYSIS OUTPUT', 'DIALYSIS TOTAL OUT','Dialysis Out', 'Dialysis Output.', 'dialysis  out','dialysis out', 'dialysis output']
Hemodialysis_Output = ['HEMODIALYSIS OUT', 'HEMODIALYSIS OUTPUT', 'Hemodialysis OUT', 'Hemodialysis Out','Hemodialysis Output', 'Hemodialysis out', 'hemodialysis out', 'hemodialysis output', 'Hemo dialysis out']
Hemodialysis = ['HEMODIALYSIS', 'HEMODIALYSIS.', 'HemoDialysis', 'Hemodialysis.', 'hemodialysis', 'hemodialysis crystal', 'hemodialysis ultrafe']
Peritoneal_Dialysis = ['PERITONEAL DIALYSIS', 'Peritoneal Dialysis', 'peritoneal dialysis', 'Peritoneal dialysis']

LDH = ['LDH','Lactate Dehydrogenase (LD)']

Temperature_C = ['Temperature', 'Temperature C', 'Temperature C (calc)', 'Temperature Celsius']
Temperature_F = ['Temperature F', 'Temperature F (calc)', 'Temperature Fahrenheit']

Creatine_Kinase = ['CK (CPK)', 'CPK','Creatine Kinase (CK)']

Auto_PEEP = ['AUTOPeeP', 'Auto PeeP', 'Auto-PEEP Level', 'Auto-PEEP level','Autopeep', 'auto-peep', 'autopeep', 'Intrinsic peep','ZAuto Peep Level']
PEEP_Set = ['PEEP Set', 'PEEP set']
Sigh_PEEP = ['PEEP Sigh level', 'SIGH PEEP LEVEL', 'Sigh PEEP level','Sigh Peep', 'Sigh peep level']
Total_PEEP = ['Total PEEP Level', 'total PeeP']

BUN = ['BUN','BUN    (6-20)','BUN (6-20)','Bun','bun','Urea Nitrogen']

Differential_Bands = ['BANDS','Bands','Bands#','Differential-Bands']

Calcium = ['CALCIUM', 'Calcium', 'Calcium, Total', 'Total Calcium', 'calcium', 'Ca','Calcium (8.4-10.2)']
Ionized_Calcium = ['% Ionized Calcium', 'IONIZED CALCIUM', 'Ion Calcium', 'Ionized Calcium','Ionized calcium', 'ionized calcium', 'Free Calcium']
Calcium_Urine = ['Calcium, Urine', 'urine calcium']

ABG_Potassium = ['ABG POTASSIUM', 'ABG Potassium']
Potassium = ['POTASSIUM', 'Potassium', 'potassium','Potassium (3.5-5.3)', 'Potassium  (3.5-5.3)']
Potassium_Urine = ['URINE POTASSIUM', 'Potassium, Urine']
Potassium_WholeBlood = ['Potassium, Whole Blood']

Creatinine = ['Cr','Creatinine','Creatinine   (0-0.7)','Creatinine (0-1.3)']
Creatinine_WholeBlood = ['Creatinine (whole blood)','Creatinine, Blood','Creatinine, Whole Blood']
Creatinine_Urine= ['Urine  Creatinine','Urine Creatinine','urine creatinine','Creatinine, Urine']

FiO2 = ['FIO2','FiO2','Inspired O2 Fraction']
FiO2_Set = ['FIO2 SET','FiO2 Set']

Glucose = ['Blood Glucose', 'BloodGlucose', 'Glucose','Glucose (70-105)']
Glucose_WholeBlood = ['Glucose (whole blood)', 'Glucose (whole blood) (soft)', 'Glucose, Whole Blood']
Fingerstick_Glucose = ['FINGERSTICK GLUCOSE.', 'Fingerstick Glucose', 'Glucose finger stick', 'finger stick glucose', 'fingerstick glucose']

HCO3 = ['BICARBONATE-HCO3', 'HCO3', 'Na Bicarbonate','Sodium Bicarbonate', 'bicarbonate', 'sodium bicarbonate', 'soduim bicarbonate', 'Bicarbonate']

Height = ['Height', 'Height (cm)']
Admission_Weight = ['Admission Weight (Kg)', 'Admit Wt']

PT = ['Prothrombin time', 'PT','PT(11-13.5)']
PTT = ['PTT','PTT(22-35)']

Anion_Gap = ['Anion Gap', 'Anion gap']

Respiratory_Rate = ["Resp Rate", "Resp. Rate", "Respiratory Rate"]
Resp_Spontaneous = ["Resp Rate (Spont)", "Spont Resp rate", "Spont. Resp. Rate", "Respiratory Rate (spontaneous)"]
Respiratory_Rate_Total = ["Resp Rate (Total)", "Respiratory Rate (Total)", "Respiratory Rate (Set)","Respiratory Rate Set"]
High_Resp_Rate = ["HIGH Resp Rate", "High Resp. Rate"]

Bilirubin_total = ["BILI", "Bili", "bili", "TBILI", "tbili", "Total Bili", "Bilirubin", "Bili level", "bili level", "Total Bilirubin", "Bilirubin, Total"]
Direct_Bilirubin = ['Direct Bili','Direct Bili (0-0.3)','Direct Bilirubin','Bilirubin, Direct']
Indirect_Bilirubin = ['Indirect Bili(0-1.0)','Bilirubin, Indirect']

WBC = ['White Blood Cells','WBC','WBC   (4-11,000)','WBC (4-11,000)','WBC 4.0-11.0','WBC Count']
RBC = ['RBC','Red Blood Cells','Red Blood C(3.6-6.2)']
GCS_EyeOpening = ['Eye Opening', 'GCS - Eye Opening']
GCS_MotorResponse = ['GCS - Motor Response', 'Motor Response']
GCS_VerbalResponse = ['GCS - Verbal Response', 'Verbal Response']

In [20]:
mapping_groups = {
    'Albumin': Albumin,
    'ALT': ALT,
    'AST': AST,
    'PLT':PLT,
    'Sodium':Sodium,
    'Sodium_WholeBlood':Sodium_WholeBlood,
    'Magnesium':Magnesium,
    'Chloride':Chloride,
    'Phosphorous':Phosphorous,
    'INR':INR,
    'NSBP':NSBP,
    'NDBP':NDBP,
    'NMBP':NMBP,
    'MAP': MAP,
    'ABP': ABP,
    'ASBP': ASBP,
    'ADBP': ADBP,
    'SaO2': SaO2,
    'SpO2': SpO2,
    'PaO2':PaO2,
    'Arterial_pH': Arterial_pH,
    'Urine_PH': Urine_PH,
    'Femoral_ArteryLine': Femoral_ArteryLine,
    'Arterial_PaCO2': Arterial_PaCO2,
    'Arterial_TCO2': Arterial_TCO2 ,
    'ETCO2': ETCO2,
    'Pain': Pain,
    'Pain_Presence': Pain_Presence,
    'Dialysis': Dialysis,
    'Dialysis_Output': Dialysis_Output,
    'Hemodialysis_Output': Hemodialysis_Output,
    'Hemodialysis': Hemodialysis,
    'Peritoneal_Dialysis': Peritoneal_Dialysis,
    'LDH': LDH,
    'Temperature_C': Temperature_C,
    'Temperature_F': Temperature_F,
    'Creatine_Kinase': Creatine_Kinase,
    'Auto_PEEP': Auto_PEEP,
    'PEEP_Set': PEEP_Set,
    'Sigh_PEEP': Sigh_PEEP,
    'Total_PEEP': Total_PEEP,
    'BUN': BUN,
    'Differential_Bands': Differential_Bands,
    'Calcium': Calcium,
    'Ionized_Calcium': Ionized_Calcium,
    'Calcium_Urine': Calcium_Urine,
    'ABG_Potassium': ABG_Potassium,
    'Potassium': Potassium,
    'Potassium_Urine': Potassium_Urine,
    'Potassium_WholeBlood':Potassium_WholeBlood,
    'Creatinine': Creatinine,
    'Creatinine_WholeBlood': Creatinine_WholeBlood,
    'Creatinine_Urine': Creatinine_Urine,
    'FiO2': FiO2,
    'FiO2_Set': FiO2_Set,
    'Glucose': Glucose,
    'Glucose_WholeBlood': Glucose_WholeBlood,
    'Fingerstick_Glucose': Fingerstick_Glucose,
    'HCO3': HCO3,
    'Height': Height,
    'Admission_Weight':Admission_Weight,
    'PT': PT,
    'PTT':PTT,
    'Anion_Gap':Anion_Gap,
    'Respiratory_Rate': Respiratory_Rate,
    'Resp_Spontaneous':Resp_Spontaneous,
    'Respiratory_Rate_Total': Respiratory_Rate_Total,
    'High_Resp_Rate': High_Resp_Rate,
    'Bilirubin_total': Bilirubin_total,
    'Direct_Bilirubin': Direct_Bilirubin,
    'Indirect_Bilirubin': Indirect_Bilirubin,
    'WBC': WBC,
    'RBC':RBC,
    'GCS_EyeOpening': GCS_EyeOpening,
    'GCS_MotorResponse': GCS_MotorResponse,
    'GCS_VerbalResponse': GCS_VerbalResponse
}

In [22]:
replace_dict = {item: key for key, values in mapping_groups.items() for item in values}
All_ITEMS['LABEL'] = All_ITEMS['LABEL'].replace(replace_dict)

In [24]:
All_ITEMS[All_ITEMS.LABEL == 'Height']

,ITEMID,LABEL,LINKSTO,CATEGORY
12300,226707,Height,chartevents,GENERAL
12304,226730,Height,chartevents,GENERAL


### 2. 整合一模一样的label

In [27]:
same = All_ITEMS[All_ITEMS.duplicated(subset=['LABEL', 'LINKSTO'],keep=False)]
same = (
    same.groupby('LABEL')
    .agg(
        final_ICD_CODE=('ITEMID', lambda x: max(x)),  # 取字符最长的ICD_CODE
        all_ICD_CODEs=('ITEMID', lambda x: list(x))            # 收集所有ICD_CODE
    )
    .reset_index()
)

# 对于完全重复的，删除重复id，保留最长的id
same_map = {
    code: final 
    for final, codes in zip(same['final_ICD_CODE'], same['all_ICD_CODEs']) 
    for code in codes if code!=final
}
All_ITEMS['ITEMID'] = All_ITEMS['ITEMID'].replace(same_map)
All_ITEMS = All_ITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
print(All_ITEMS.shape)

(14167, 4)


In [29]:
len(pd.unique(All_ITEMS.ITEMID))

14167

In [31]:
All_ITEMS[All_ITEMS.LABEL == 'Height']

,ITEMID,LABEL,LINKSTO,CATEGORY
12300,226730,Height,chartevents,GENERAL


In [33]:
same_map

{80219: 80265,
 51829: 51925,
 3725: 4194,
 2293: 6156,
 2968: 6156,
 6145: 6156,
 5981: 6156,
 5600: 6156,
 2374: 6156,
 2637: 6156,
 8555: 225310,
 8368: 225310,
 220051: 225310,
 769: 220644,
 50861: 220644,
 24: 224404,
 6701: 225309,
 51: 225309,
 220050: 225309,
 770: 220587,
 50878: 220587,
 27: 224003,
 51133: 53157,
 52769: 53157,
 53159: 229355,
 52075: 229355,
 52076: 52316,
 31: 224084,
 229319: 229321,
 32: 224086,
 762: 226512,
 40: 223836,
 53138: 227456,
 51775: 227456,
 51910: 227456,
 53116: 227456,
 51835: 227456,
 51926: 227456,
 51807: 227456,
 51011: 227456,
 51776: 227456,
 51836: 227456,
 40548: 227456,
 2358: 227456,
 46564: 227456,
 53085: 227456,
 1521: 227456,
 50862: 227456,
 45403: 227456,
 3066: 227456,
 51069: 52703,
 53139: 53140,
 50863: 53086,
 3728: 53086,
 51837: 53141,
 51808: 53141,
 51928: 53141,
 51961: 53141,
 51778: 53141,
 53117: 53141,
 51911: 53141,
 51838: 53143,
 51912: 53143,
 51809: 53143,
 51929: 53143,
 51962: 53143,
 53118: 53143,
 5

# CHARTEVENTS + LABEVENTS

In [37]:
#CHARTEVENTS
dataset_path = 'C:/MIMIC IV/3.1/icu/chartevents.csv.gz'

chunksize = 10000000
Chart = pd.DataFrame()
# 遍历分块的 CSV 数据
for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):
    # 选择感兴趣的列
    chunk.columns = chunk.columns.str.upper()
    chunk.rename(columns={'STAY_ID': 'ICUSTAY_ID'}, inplace=True)
    chunk = chunk[['ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]
    
    # 过滤 stay_id 和 itemid
    chunk = chunk[chunk['ICUSTAY_ID'].isin(df['ICUSTAY_ID'])]
    
    # 保留 'value' 或 'valuenum' 非空的行
    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    
    # 检查是否有行数据
    if chunk.shape[0] > 0:
        # 合并结果
        Chart = pd.concat([Chart, chunk], ignore_index=True)
del(chunk)
print(Chart.shape, len(Chart.ITEMID.unique()))

44it [07:41, 10.49s/it]


(351111895, 5) 2308


In [38]:
# LABEVENTS
dataset_path = 'C:/MIMIC IV/3.1/hosp/labevents.csv.gz'

chunksize = 10000000
Lab = pd.DataFrame()
# 遍历分块的 CSV 数据
for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):
    # 选择感兴趣的列
    chunk.columns = chunk.columns.str.upper()
    chunk = chunk[['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]
    
    # 过滤 stay_id 和 itemid
    chunk = chunk[chunk['HADM_ID'].isin(df['HADM_ID'])]
    
    # 保留 'value' 或 'valuenum' 非空的行
    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    
    # 检查是否有行数据
    if chunk.shape[0] > 0:
        # 合并结果
        Lab = pd.concat([Lab, chunk], ignore_index=True)
del(chunk)        
print(Lab.shape, len(Lab.ITEMID.unique()))

16it [03:03, 11.47s/it]


(32875512, 5) 778


In [39]:
Chart['ICUSTAY_ID'] = Chart['ICUSTAY_ID'].astype(int)
Lab['HADM_ID'] = Lab['HADM_ID'].astype(int)
print(Chart.shape, Lab.shape)

Chart = Chart.dropna(subset=['VALUENUM'], how='all')
Lab = Lab.dropna(subset=['VALUENUM'], how='all')
print(Chart.shape, Lab.shape)

(351111895, 5) (32875512, 5)
(141054270, 5) (31692992, 5)


In [40]:
Chart_keep = Chart.copy()
LAB_keep = Lab.copy()

In [41]:
def pre_icu_ts_data(df):
    # 仅保留 ICU 内的记录
    within_criteria = df['CHARTTIME'].between(df['INTIME'], df['OUTTIME'])
    df = df[within_criteria].copy()

    # 计算距离 ICU 入科的时间
    df['event_time_from_admit'] = df['CHARTTIME'] - df['INTIME']
    df['start_time'] = df['event_time_from_admit'].dt.total_seconds() / 3600
    df['los'] = (df['OUTTIME'] - df['INTIME']).dt.total_seconds() / 3600

    # 删除负值
    df = df[df['start_time'] >= 0]

    # 保留需要的列
    cols_to_keep = ['ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM',
                    'INTIME', 'OUTTIME', 'event_time_from_admit', 'start_time', 'los']
    df = df[cols_to_keep]

    return df

In [42]:
# 先把 LAB 和 CHART 处理成可以合并在一起的数据
LAB_keep = pd.merge(LAB_keep,df[['HADM_ID','ICUSTAY_ID','INTIME', 'OUTTIME']],on='HADM_ID',how='left')
LAB_keep['INTIME'] = pd.to_datetime(LAB_keep['INTIME'])
LAB_keep['OUTTIME'] = pd.to_datetime(LAB_keep['OUTTIME'])
LAB_keep['CHARTTIME'] = pd.to_datetime(LAB_keep['CHARTTIME'])
print(LAB_keep.shape)

#仅保留在ICU期间的记录
LAB_keep = pre_icu_ts_data(LAB_keep)
print(LAB_keep.shape)

(31692992, 8)
(16918758, 10)


In [43]:
Chart_keep = pd.merge(Chart_keep,df[['ICUSTAY_ID','INTIME', 'OUTTIME']],on='ICUSTAY_ID',how='left')
Chart_keep['INTIME'] = pd.to_datetime(Chart_keep['INTIME'])
Chart_keep['OUTTIME'] = pd.to_datetime(Chart_keep['OUTTIME'])
Chart_keep['CHARTTIME'] = pd.to_datetime(Chart_keep['CHARTTIME'])
print(Chart_keep.shape)

#仅保留在ICU期间的记录
Chart_keep = pre_icu_ts_data(Chart_keep)
print(Chart_keep.shape)

(141054270, 7)
(139940040, 10)


In [44]:
All_TS = pd.concat([Chart_keep,LAB_keep])
print(Chart_keep.shape,LAB_keep.shape,All_TS.shape)
del(Chart, Lab)

(139940040, 10) (16918758, 10) (156858798, 10)


In [98]:
All_TS['ITEMID'] = All_TS['ITEMID'].replace(same_map)
print(All_TS.shape)
All_TS = All_TS.drop_duplicates(keep='first')
print(All_TS.shape)

(152096707, 11)
(152096707, 11)


In [99]:
All_TS.head()

,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUENUM,INTIME,OUTTIME,event_time_from_admit,start_time,los,LABEL
0,37081114,2150-11-04 12:00:00,223834,12,12.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 16:23:00,40.383333,93.438056,O2 Flow
1,37081114,2150-11-04 12:00:00,223835,50,50.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 16:23:00,40.383333,93.438056,Inspired O2 Fraction
2,37081114,2150-11-04 12:00:00,227287,4,4.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 16:23:00,40.383333,93.438056,O2 Flow (additional cannula)
3,37081114,2150-11-04 18:00:00,223834,12,12.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 22:23:00,46.383333,93.438056,O2 Flow
4,37081114,2150-11-04 18:00:00,223835,70,70.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 22:23:00,46.383333,93.438056,Inspired O2 Fraction


In [47]:
All_TS = pd.merge(All_TS,All_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
print(All_TS.shape,len(All_TS.ITEMID.unique()))

(152096707, 11) 1430


In [52]:
## 转换单位
# Admission Weight (lbs.) - 226531
All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'VALUENUM'] = (All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'VALUENUM'] * 0.453592).round(2)
All_TS.loc[All_TS['ITEMID'] == 226531, 'ITEMID'] = 226512
All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'LABEL'] = 'Admission_Weight'

# Temperature_F - 223761
All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'VALUENUM'] = ((All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'VALUENUM'] - 32) * 5/9).round(2)
All_TS.loc[All_TS['ITEMID'] == 223761, 'ITEMID'] = 223762
All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'LABEL'] = 'Temperature_C'

In [ ]:
# All_TS.to_csv('C:/Users/cheny/Desktop/UTI_ML/wln/can_delete/within_All_TS_IV.csv',index=False)

In [35]:
All_TS = pd.read_csv('C:/Users/cheny/Desktop/UTI_ML/wln/can_delete/within_All_TS_IV.csv')
print(All_TS.shape,len(All_TS.ITEMID.unique()))

(152096707, 11) 1428


In [36]:
# 查看缺失率大于30%删除
unique_C = All_TS[['ICUSTAY_ID', 'ITEMID']].drop_duplicates()
Citemid_frequency = unique_C.groupby('ITEMID')['ICUSTAY_ID'].count().reset_index()
Citemid_frequency.columns = ['ITEMID', 'ICUSTAY_ID_count']
Citemid_frequency = Citemid_frequency.sort_values(by='ICUSTAY_ID_count', ascending=False)
print(Citemid_frequency.shape)
Citemid_frequency = Citemid_frequency[Citemid_frequency.ICUSTAY_ID_count >= len(All_TS.ICUSTAY_ID.unique()) * 0.3]
Citemid_frequency = pd.merge(Citemid_frequency,All_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
print(Citemid_frequency.shape)

(1428, 2)
(190, 3)


In [37]:
# 警报类（Alarm）特征（由原始生理值派生，信息冗余）
# ADT ['Date of Birth', 'Discharge Date/Time', 'Expected Discharge Date','Gender', 'Hospital Admit Date', 'ICU Admission date', 'Language','Marital Status', 'Patient Location', 'Race', 'Religion']
ADTs = [226515, 226516, 226518, 226228, 226724, 220003, 226544, 226381,227378, 226545, 226543]

alarm_features_to_remove = [
    "Alarms On",'HR Alarm [High]','HR Alarm [Low]',
    'NBP Alarm [High]','NBP Alarm [Low]',
    'Resp Alarm [High]','Resp Alarm [Low]',
    'SpO2 Alarm [High]','SpO2 Alarm [Low]',
    'Heart Rate Alarm - Low',
    'Heart rate Alarm - High',
    'O2 Saturation Pulseoxymetry Alarm - Low',
    'O2 Saturation Pulseoxymetry Alarm - High',
    'Resp Alarm - Low',
    'Resp Alarm - High',
    'Non-Invasive Blood Pressure Alarm - Low',
    'Non-Invasive Blood Pressure Alarm - High',
    'Minute Volume Alarm - High',
    'Minute Volume Alarm - Low',
    'Arterial Blood Pressure Alarm - Low',
    'Arterial Blood Pressure Alarm - High',
    "PSV Level", 
    'Paw High',
    'Vti High',
    'Fspn High',
    'H', 'L', 'I'
]
# 主观或社会心理类特征（难以量化或与预后无强相关
subjective_features_to_remove = [
    "Richmond-RAS Scale", "Goal Richmond-RAS Scale",
    "Acuity Workload Question 1", "Acuity Workload Question 2", 
    'Is the spokesperson the Health Care Proxy',
    'Any fear in relationships',
    'Sexuality / reproductive problems',
    'Emotional / physical / sexual harm by partner or close relation',
    'Unable to assess psychological',
    'Unable to assess nutrition / education',
    'Social work consult',
    "History of slips / falls", "Home TF",
    "Recreational drug use",  'Orientation',
    "Special diet", "PH (dipstick)", "Specific Gravity (urine)",
    "Braden Sensory Perception", "Braden Mobility", "Braden Moisture",
    "Braden Nutrition", "Braden Activity", "Braden Friction/Shear",
    # 'Strength R Arm', 'Strength L Arm',
    # 'Strength R Leg', 'Strength L Leg',     # 主观评估项，重复性与可靠性较差
    "Activity / Mobility (JH-HLM)", "Gait/Transferring", "Ambulatory aid",
    #"Mental status",
    # "Visual / hearing deficit","Unintentional weight loss >10 lbs.","History of falling (within 3 mnths)",
]
# 护理操作/监控设备类（非病人生理状态）
procedural_features_to_remove = [
    'Insulin pump', 'Cough/Deep Breath','Apnea Interval','Cuff Pressure',
    'Intravenous  / IV access prior to admission','Cuff Pressure','Ventilator Mode',
    'Parameters Checked',
    'ST Segment Monitoring On',
    'IV/Saline lock',
    '20 Gauge placed in outside facility',
    '20 Gauge placed in the field',
    '20 Gauge Dressing Occlusive',
    '18 Gauge Dressing Occlusive',
    '18 Gauge placed in outside facility',
    '18 Gauge placed in the field',
    'Multi Lumen placed in outside facility',
    'Arterial Line placed in outside facility',
    'Arterial Line Zero/Calibrate',
    'Ventilator Tank #1',
    'Ventilator Tank #2',
    'Called Out',
    "Subglottal Suctioning", "Eye Care",'High risk (>51) interventions',
    'Secondary diagnosis',"Pressure Ulcer Present",
    "Differential-Basos","Differential-Monos", "Differential-Eos"
]
# 重复单位
redundant_unit_features_to_remove = [
    'Glucose finger stick (range 70-100)',
    'Tidal Volume (set)','Tidal Volume (spontaneous)','Minute Volume', # 可由 tidal volume × RR 得出
]

Chart_features_to_remove = (
    alarm_features_to_remove +
    subjective_features_to_remove +
    procedural_features_to_remove +
    redundant_unit_features_to_remove
)
len(Chart_features_to_remove)

89

In [38]:
Citemid_frequency = Citemid_frequency[~(Citemid_frequency.LABEL.isin(Chart_features_to_remove))]
Citemid_frequency = Citemid_frequency[~(Citemid_frequency.ITEMID.isin(ADTs))]
Citemid_frequency.shape

(112, 3)

In [39]:
print(len(pd.unique(Citemid_frequency.LABEL.values)),list(Citemid_frequency.ITEMID.values))

112 [220045, 228232, 220739, 223900, 223901, 220210, 223762, 226253, 220615, 220645, 225624, 220602, 227443, 227442, 227073, 220546, 220228, 227457, 227346, 220545, 227341, 220621, 52623, 52535, 52569, 50882, 52610, 52028, 51493, 51691, 53186, 53185, 51277, 220181, 220179, 220180, 220635, 225677, 50970, 225625, 52035, 227465, 227467, 51675, 227466, 227881, 226512, 225664, 223834, 229326, 224409, 52730, 228412, 228409, 228411, 228410, 220224, 52038, 52040, 50804, 225668, 53154, 225092, 224639, 223835, 220644, 220587, 225690, 53086, 225612, 53166, 225106, 223830, 225698, 220235, 224828, 229323, 225118, 225087, 225126, 225113, 225667, 220339, 224685, 224690, 224695, 224689, 53188, 224697, 52272, 52256, 225312, 225310, 225309, 52225, 53164, 225641, 225643, 224738, 225124, 224696, 227456, 220632, 229325, 224700, 227464, 52452, 51994, 228388, 220274, 227429, 225634]


In [40]:
Citemid_frequency[Citemid_frequency.LABEL.duplicated(keep=False)]

,ITEMID,ICUSTAY_ID_count,LABEL


In [41]:
iiis = [228232, 223762, 220045, 220615, 220739, 223901, 225624, 223900, 220210, 52610, 52569, 52028, 220228, 52623, 227457, 220546, 52535, 50882, 51493, 53185, 53186, 51691, 51277, 227073, 220179, 220180, 220181, 220635, 50970, 52035, 225677, 227465, 227466, 227467, 51675, 226512, 52730, 225664, 220224, 50804, 52038, 52040, 220235, 224828, 224639, 787, 742, 198, 87, 225312, 225309, 225310, 53154, 220339, 224690, 224697, 225667, 224695, 224689, 225668, 220602, 227443, 220545, 220645, 227442, 220621, 470, 225690, 220644, 220587, 225625, 224696, 53086, 223834, 225634, 227346, 227341, 226253, 227456, 52452, 227881, 581, 227688, 51994, 53188, 52272, 225641, 52256, 4753, 52225, 53164, 777, 226861, 224409, 225187, 225185, 580, 220632, 7570, 50911, 50816, 225313, 450, 50810, 434, 218, 631, 578, 52455]
len(iiis)

109

In [42]:
iviiis = list(pd.unique(list(Citemid_frequency.ITEMID.values)+iiis))
iviiis = list(set(iviiis)-set([578, 450, 580, 581, 198, 742, 227688, 777, 4753, 7570, 787, 434, 87, 631, 470, 218,229323, 53166]))
print(len(iviiis),iviiis)

118 [52225, 227341, 223762, 227346, 220179, 220181, 220180, 225309, 225310, 52256, 225312, 225313, 228388, 227881, 226861, 52272, 220210, 228409, 228410, 228411, 228412, 224828, 220224, 220739, 220228, 220235, 51277, 223830, 223834, 223835, 227429, 227442, 227443, 50804, 220274, 50810, 227456, 227457, 50816, 227464, 227465, 227466, 227467, 224409, 223900, 223901, 220339, 50882, 226512, 50911, 52452, 52455, 227073, 50970, 51994, 51493, 52535, 52028, 225087, 52035, 225092, 52038, 52040, 225612, 225106, 225624, 52569, 225625, 225113, 53086, 225118, 225634, 225124, 225126, 225641, 225643, 224639, 225664, 220545, 220546, 52610, 225668, 225667, 228232, 220045, 225677, 52623, 225690, 225185, 53154, 225698, 225187, 220587, 53164, 224685, 224689, 224690, 224695, 224696, 224697, 220602, 224700, 53185, 53186, 53188, 220615, 226253, 220621, 229326, 229325, 220632, 220635, 51675, 224738, 220644, 220645, 51691, 52730]


In [43]:
All_ITEMS[All_ITEMS.ITEMID.isin(iviiis)].LABEL.values

array(['Ionized_Calcium', 'Albumin', 'ALT', 'SaO2', 'ADBP', 'MAP', 'ASBP',
       'AST', 'Admission_Weight', 'Alkaline Phosphatase',
       'Alkaline Phosphate', 'Anion_Gap', 'Arterial Base Excess',
       'Arterial_PaCO2', 'PaO2', 'HCO3', 'Bilirubin_total', 'BUN',
       'Back Care', 'Base Excess', 'Basophils', 'Bed Bath', 'Glucose',
       'Calcium', 'CAM-ICU Inattention', 'CAM-ICU MS Change',
       'Creatine_Kinase', 'Calcium non-ionized', 'Calculated Total CO2',
       'Chloride', 'Chloride (serum)', 'Creatinine',
       'Creatine Kinase, MB Isoenzyme', 'Pain_Presence', 'Daily Weight',
       'Dialysis patient', 'Differential-Lymphs', 'Differential-Neuts',
       'Difficulty swallowing', 'ETOH', 'Eosinophils', 'GCS_EyeOpening',
       'Fingerstick_Glucose', 'FiO2', 'GCS_MotorResponse',
       'GCS_VerbalResponse', 'Glucose (serum)', 'Glucose_WholeBlood',
       'HCO3 (serum)', 'Heart Rate', 'Hematocrit', 'Hematocrit (serum)',
       'Hematocrit, Calculated', 'Hemoglobin',
       '

In [44]:
All_TS_keep = All_TS[All_TS.ITEMID.isin(iviiis)]
print(All_TS_keep.shape, len(All_TS_keep.ITEMID.unique()))

(89563961, 11) 118


In [45]:
print(list(All_TS_keep.ITEMID.unique()))

[223834, 223835, 220045, 220179, 220180, 220181, 220210, 226512, 226253, 227881, 220739, 223900, 223901, 227341, 227346, 228232, 223762, 225185, 225187, 225087, 225092, 225106, 225113, 225118, 225126, 225664, 220228, 220545, 220546, 227457, 220602, 220615, 220621, 220635, 220645, 225624, 225625, 225634, 225677, 227073, 227442, 227443, 227429, 228409, 228410, 228411, 228412, 224409, 229326, 227465, 227466, 227467, 224639, 225124, 225641, 225643, 220339, 224685, 224690, 224689, 224695, 224696, 224697, 224738, 224700, 229325, 220587, 220644, 225612, 225690, 227456, 220274, 225667, 225668, 228388, 227464, 220224, 226861, 220235, 223830, 224828, 225698, 225309, 225310, 225312, 220632, 225313, 52028, 53186, 53185, 51691, 51277, 51493, 50882, 52035, 52535, 52569, 50970, 52610, 52623, 52730, 51994, 51675, 52225, 52256, 53188, 52272, 53164, 53086, 50911, 52038, 50804, 50810, 53154, 52040, 52452, 52455, 50816]


In [46]:
#del(Chart_keep,LAB_keep)
del(Citemid_frequency)

In [47]:
#创建ID和文字信息对应的字典
included_itme = All_ITEMS[All_ITEMS.ITEMID.isin(iviiis)]
c_dict = dict(zip(included_itme['ITEMID'], included_itme['LABEL']))
print(All_TS_keep.shape,len(All_TS_keep.ITEMID.unique()))
print(len(c_dict),c_dict)

(89563961, 11) 118
118 {225667: 'Ionized_Calcium', 227456: 'Albumin', 220644: 'ALT', 226861: 'SaO2', 225310: 'ADBP', 225312: 'MAP', 225309: 'ASBP', 220587: 'AST', 226512: 'Admission_Weight', 53086: 'Alkaline Phosphatase', 225612: 'Alkaline Phosphate', 227073: 'Anion_Gap', 224828: 'Arterial Base Excess', 220235: 'Arterial_PaCO2', 220224: 'PaO2', 50882: 'HCO3', 225690: 'Bilirubin_total', 225624: 'BUN', 225187: 'Back Care', 52038: 'Base Excess', 52225: 'Basophils', 225313: 'Bed Bath', 52569: 'Glucose', 52035: 'Calcium', 229325: 'CAM-ICU Inattention', 229326: 'CAM-ICU MS Change', 225634: 'Creatine_Kinase', 225625: 'Calcium non-ionized', 50804: 'Calculated Total CO2', 52535: 'Chloride', 220602: 'Chloride (serum)', 220615: 'Creatinine', 50911: 'Creatine Kinase, MB Isoenzyme', 225113: 'Pain_Presence', 224639: 'Daily Weight', 225126: 'Dialysis patient', 225641: 'Differential-Lymphs', 225643: 'Differential-Neuts', 225118: 'Difficulty swallowing', 225106: 'ETOH', 52256: 'Eosinophils', 220739: 'G

In [48]:
# 判断feature的类型，分类型还是数值型
def is_numeric(value):
    try:
        float(value)  # 尝试将值转换为浮点数
        return True
    except (ValueError, TypeError):
        return False

#异常值处理
def compute_outlier_imputation(arr, upper_percentile, lower_percentile):
    
    arr = pd.to_numeric(arr)
    # 计算上下限阈值
    upper_bound = np.percentile(arr, upper_percentile)
    lower_bound = np.percentile(arr, lower_percentile)
    
    # 替换异常值
    arr[arr < lower_bound] = lower_bound
    arr[arr > upper_bound] = upper_bound

    return arr

def outlier_imputation(data, id_attribute, value_attribute, num_col, upper_percentile=98, lower_percentile=2):
    # 按指定的分组列进行分组
    grouped = data.groupby(id_attribute)

    # 遍历每个分组
    for id_number, group in grouped:
        # 仅处理 num_col 中的 id_attribute
        if id_number not in num_col:
            continue
        
        index = group.index
        values = group[value_attribute].values

        # 调用异常值处理函数
        imputed_values = compute_outlier_imputation(values, upper_percentile, lower_percentile)
        
        # 替换处理后的值
        data.loc[index, value_attribute] = imputed_values

    # 删除因异常值处理引入的 NaN
    data = data.dropna(subset=[value_attribute])

    return data

In [49]:
All_TS_keep.head(2)

,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUENUM,INTIME,OUTTIME,event_time_from_admit,start_time,los,LABEL
0,37081114,2150-11-04 12:00:00,223834,12,12.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 16:23:00,40.383333,93.438056,O2 Flow
1,37081114,2150-11-04 12:00:00,223835,50,50.0,2150-11-02 19:37:00,2150-11-06 17:03:17,1 days 16:23:00,40.383333,93.438056,Inspired O2 Fraction


In [50]:
All_TS_keep['is_numeric'] = All_TS_keep['VALUENUM'].apply(is_numeric)

result = All_TS_keep.groupby('ITEMID')['is_numeric'].all().reset_index()
result.rename(columns={'is_numeric': 'is_fully_numeric'}, inplace=True)

num_col = result[result.is_fully_numeric==True].ITEMID.values
print(len(All_TS_keep.ITEMID.value_counts()),len(num_col))

All_TS_keep = outlier_imputation(All_TS_keep, 'ITEMID', 'VALUENUM', num_col, upper_percentile=98, lower_percentile=2)

118 118


In [51]:
All_TS_keep.shape

(89563961, 12)

In [52]:
del(All_TS)

In [53]:
los_dict = All_TS_keep.groupby('ICUSTAY_ID')['los'].first().to_dict()
max_los = max(los_dict.values())
print('所有人中最大ICU LOS：', max_los, max_los*4)
 
final_TS=pd.DataFrame()
t=0

for i in tqdm(np.arange(0, max_los, 0.25)):  # 每15分钟一个间隔
    # 根据时间段筛选数据并按 'stay_id' 和 'itemid' 分组，计算均值
    sub_TS = All_TS_keep[(All_TS_keep['start_time'] >= i) & (All_TS_keep['start_time'] < i + 0.25)]
    
    # 对于数值型 itemid，计算中位数来代表每15分钟的值
    sub_TS_num = sub_TS[sub_TS['ITEMID'].isin(num_col)].groupby(['ICUSTAY_ID', 'ITEMID']).agg({'VALUENUM': np.nanmedian}).reset_index()
    sub_TS = sub_TS_num

    # 添加时间信息
    sub_TS['start_time'] = t

    # 将结果添加到 final_TS
    if final_TS.empty:
        final_TS = sub_TS
    else:
        final_TS = pd.concat([final_TS, sub_TS], ignore_index=True)

    t += 1

所有人中最大ICU LOS： 3832.0033333333336 15328.013333333334


100%|██████████████████████████████████████████████████████████████████████████| 15329/15329 [2:45:58<00:00,  1.54it/s]


In [72]:
# final_TS.to_csv('C:/Users/cheny/Desktop/UTI_ML/wln/can_delete/within_All_TS_keep_IV.csv',index=False)

In [54]:
final_TS['ITEMID'] = final_TS['ITEMID'].map(c_dict)
print(final_TS.ITEMID.value_counts())

ITEMID
Heart Rate                       7027690
SpO2                             6999759
Respiratory_Rate                 6938744
NSBP                             4338628
NDBP                             4337839
                                  ...   
Troponin-T                         53785
Bed Bath                           53043
Creatine_Kinase                    48621
Creatine Kinase, MB Isoenzyme      45405
Specific Gravity                   41347
Name: count, Length: 118, dtype: int64


In [55]:
final_TS.head()

,ICUSTAY_ID,ITEMID,VALUENUM,start_time
0,30000153,Respiratory_Rate,18.0,0
1,30000213,Heart Rate,74.0,0
2,30000213,NSBP,168.0,0
3,30000213,NDBP,65.0,0
4,30000213,NMBP,95.0,0


In [56]:
print(len(final_TS.ITEMID.unique()),list(final_TS.ITEMID.unique()))

118 ['Respiratory_Rate', 'Heart Rate', 'NSBP', 'NDBP', 'NMBP', 'PEEP_Set', 'FiO2', 'Tidal Volume (observed)', 'Resp_Spontaneous', 'Respiratory_Rate_Total', 'Peak Insp. Pressure', 'Mean Airway Pressure', 'Inspiratory Time', 'SpO2', 'GCS_EyeOpening', 'O2 Flow', 'GCS_VerbalResponse', 'GCS_MotorResponse', 'Admission_Weight', 'History of falling (within 3 mnths)', 'Mental status', 'Calculated Total CO2', 'HCO3', 'Creatine Kinase, MB Isoenzyme', 'Phosphate', 'RDW', 'RBC', 'INR(PT)', 'MCV', 'Hematocrit', 'Calcium', 'Base Excess', 'pCO2', 'Basophils', 'Eosinophils', 'Monocytes', 'Chloride', 'Glucose', 'Potassium', 'Sodium', 'pH', 'Alkaline Phosphatase', 'Lactate', 'Neutrophils', 'MCHC', 'MCH', 'Lymphocytes', 'PaO2', 'Hemoglobin', 'PH (Venous)', 'Hematocrit (serum)', 'WBC', 'AST', 'Chloride (serum)', 'Creatinine', 'Glucose (serum)', 'Magnesium', 'ALT', 'Sodium (serum)', 'Temperature_C', 'Alkaline Phosphate', 'BUN', 'Calcium non-ionized', 'Differential-Lymphs', 'Differential-Neuts', 'Lactic Acid

In [57]:
final_TS[final_TS.ITEMID.isin(['GCS_EyeOpening','GCS_VerbalResponse', 'GCS_MotorResponse'])].head()

,ICUSTAY_ID,ITEMID,VALUENUM,start_time
20,30000646,GCS_EyeOpening,4.0,0
22,30000646,GCS_VerbalResponse,5.0,0
23,30000646,GCS_MotorResponse,6.0,0
226,30003372,GCS_EyeOpening,4.0,0
229,30003372,GCS_VerbalResponse,5.0,0


In [58]:
def create_Dict(TS):
    dataDic = {}
    feat = list(TS['ITEMID'].unique())  # 获取所有 itemid
    feat = feat + ['GCS_Total']
    
    for hid in tqdm(TS.ICUSTAY_ID.unique()):  # 遍历每个 ICUSTAY_ID
        df2 = TS[TS['ICUSTAY_ID'] == hid]  # 获取对应 stay_id 的数据
        
        if df2.shape[0] == 0:
            print(f'空：{hid}')
            break

        else:
            # 按 start_time 和 itemid 对数据进行透视
            val = df2.pivot(index='start_time', columns='ITEMID', values='VALUENUM')
            
            add_indices = pd.Index(range(df2.start_time.max())).difference(val.index)
            add_df = pd.DataFrame(index=add_indices, columns=val.columns)
            val = pd.concat([val, add_df])  # 合并原数据和补全的数据
            val = val.sort_index()  # 按时间索引排序

            # 添加缺失的 itemid 列
            feat_df = pd.DataFrame(columns=list(set(feat) - set(val.columns)))  # 找到缺失的 itemid 列
            val = pd.concat([val, feat_df], axis=1)  # 将缺失的列添加到 val 中

            # 特别处理GCS-Total
            val['GCS_Total'] = val[['GCS_EyeOpening', 'GCS_VerbalResponse', 'GCS_MotorResponse']].sum(axis=1, skipna=True)
            # 因为 GCS 是单独处理的，其实0是空
            val['GCS_Total'] = val['GCS_Total'].replace(0, np.nan)
            
            val = val[feat]  # 确保列的顺序与原始 feat 相同

        # 保存数据到 CSV
        val.to_csv(f'C:/Users/cheny/Desktop/UTI_ML/wln/MIMIC4_TS/CHARTS_IV/{hid}.csv', index=False)

In [59]:
#为每个病人存储ICU的时序数据
create_Dict(final_TS)

100%|██████████████████████████████████████████████████████████████████████████| 67207/67207 [1:13:54<00:00, 15.16it/s]


# TS static concat

In [62]:
import os

In [63]:
TS_file = os.listdir('C:/Users/cheny/Desktop/UTI_ML/wln/MIMIC4_TS/CHARTS_IV')
print(len(TS_file), TS_file[0])

67207 30000153.csv


In [64]:
df.shape

(67207, 373)

In [65]:
ts_median = pd.DataFrame()

if not TS_file:
    print("当前文件夹中没有找到CSV文件")
else:
    # 处理每个CSV文件
    for csv_file in tqdm(TS_file):
        try:
            # 读取CSV文件
            df = pd.read_csv(f'C:/Users/cheny/Desktop/UTI_ML/wln/MIMIC4_TS/CHARTS_IV/{csv_file}')

            # # 因为 GCS 是单独处理的，其实0是空，所以反工一下！如果之前处理对了就不用
            # df['GCS_Total'] = df['GCS_Total'].replace(0, np.nan)
            
            # 只取前192行
            if len(df) > 192:
                df = df.iloc[:192]
            
            # 计算每列的中位数
            medians = {}
            for column in df.columns:
                # 检查列是否为空
                if df[column].notna().any():  # 如果列不是全空
                    medians[column] = df[column].median()
                else:
                    medians[column] = np.nan  # 空列则为空值
            
            # 将结果转换为DataFrame以便于查看
            result_df = pd.DataFrame([medians])
            result_df['ICUSTAY_ID'] = int(csv_file.split('.')[0])
            ts_median = pd.concat([ts_median,result_df])
            
        except Exception as e:
            print(f"处理文件 {csv_file} 时出错: {e}\n")

100%|████████████████████████████████████████████████████████████████████████████| 67207/67207 [28:26<00:00, 39.39it/s]


In [66]:
print(ts_median.shape)

(67207, 120)


In [67]:
ts_median.describe()

,Respiratory_Rate,Heart Rate,NSBP,NDBP,NMBP,PEEP_Set,FiO2,Tidal Volume (observed),Resp_Spontaneous,Respiratory_Rate_Total,...,Pain_Presence,Difficulty swallowing,Unintentional weight loss >10 lbs.,LDH,Creatine_Kinase,Pain Level Response,Dialysis patient,Oxygen,GCS_Total,ICUSTAY_ID
count,67075.000000,67153.000000,64736.000000,64725.000000,64731.000000,30988.000000,36497.000000,30851.000000,30624.000000,30779.000000,...,32329.000000,33348.000000,28344.000000,23288.000000,18581.000000,44101.000000,32377.000000,11208.000000,67137.000000,6.720700e+04
mean,18.925237,83.616614,117.808171,64.168185,77.856139,6.080323,49.673418,470.699684,6.467828,18.348346,...,0.416762,0.123995,0.122945,470.905048,958.012459,1.899056,0.058174,55.023733,12.815839,3.498472e+07
std,3.829684,15.067416,17.045419,11.573688,11.808693,2.317801,15.237154,94.823008,8.609636,4.493918,...,0.492544,0.329137,0.327966,756.744056,3207.077333,1.947331,0.234026,20.000648,3.335553,2.889007e+06
min,10.000000,53.000000,82.000000,38.000000,52.000000,0.000000,30.000000,200.000000,0.000000,10.000000,...,0.000000,0.000000,0.000000,134.000000,17.000000,0.000000,0.000000,30.000000,1.000000,3.000015e+07
25%,16.000000,73.000000,105.000000,56.000000,69.000000,5.000000,40.000000,409.000000,0.000000,15.000000,...,0.000000,0.000000,0.000000,202.500000,68.000000,0.000000,0.000000,40.000000,11.000000,3.248567e+07
50%,18.000000,82.000000,115.750000,63.000000,76.500000,5.000000,50.000000,466.000000,0.000000,17.500000,...,0.000000,0.000000,0.000000,275.000000,162.000000,2.000000,0.000000,50.000000,15.000000,3.497759e+07
75%,21.000000,93.000000,129.000000,71.000000,85.000000,5.000000,50.000000,524.000000,13.000000,20.000000,...,1.000000,0.000000,0.000000,415.000000,530.000000,3.000000,0.000000,60.000000,15.000000,3.747504e+07
max,34.000000,128.000000,171.000000,103.000000,116.000000,16.000000,100.000000,798.000000,34.000000,35.000000,...,1.000000,1.000000,1.000000,6685.000000,32692.000000,8.000000,1.000000,100.000000,15.000000,3.999986e+07


In [68]:
ts_median['GCS_Total'].value_counts()

GCS_Total
15.00    36300
14.00     6802
10.00     4541
3.00      2590
11.00     2583
13.00     2234
9.00      1954
8.00      1655
7.00      1624
14.50     1285
6.00      1050
12.00      930
12.50      577
10.50      447
5.00       389
4.00       364
9.50       355
13.50      341
8.50       304
7.50       193
11.50      189
6.50       161
4.50        87
3.50        83
5.50        74
2.00        11
10.75        3
9.75         2
9.25         2
12.25        1
2.50         1
1.00         1
8.75         1
12.75        1
6.25         1
8.25         1
Name: count, dtype: int64

In [69]:
# 因为有些ICU前48小时是没有值的后面才有，所以这一不要删掉前48小时没有值的
ts_median = ts_median[~(ts_median[ts_median.drop('ICUSTAY_ID', axis=1).columns].isna().all(axis=1))]
ts_median.shape

(67207, 120)

In [70]:
ts_median.to_csv('C:/Users/cheny/Desktop/UTI_ML/wln/can_delete/IV_ts_static.csv',index=False)